In [1]:
import numpy as np
import subprocess
import tqdm
import pandas as pd

import os
import os.path as osp

import glob

import h5py
import uproot

import torch
from torch import nn
from torch_geometric.data import Data
from torch_geometric.data import Dataset
from torch_geometric.data import DataLoader

import awkward as ak
import random

In [2]:

v_path = '/vols/cms/mm1221/Data/2e/val/raw/val.root'

In [3]:
import uproot
import matplotlib.pyplot as plt

# Open files
v_file = uproot.open(v_path)

# Inspect contents
print("Electron file contents:")
print(v_file.keys())


Electron file contents:
['tracksters;1', 'candidates;1', 'trackstersMerged;1', 'associations;1', 'simtrackstersSC;1', 'simtrackstersCP;1', 'tracks;1', 'simTICLCandidate;1', 'clusters;1']


In [4]:
# Loop through all keys and inspect
for key in v_file.keys():
    print(f"Inspecting key: {key}")
    obj = v_file[key]
    
    # Check if the object is a tree or a nested directory
    if hasattr(obj, "keys"):  # If it has keys, it can be expanded further
        print("Subkeys (branches or nested objects):")
        print(obj.keys())
    else:
        print(f"{key} is not a tree or expandable object.")
    print("-" * 50)

Inspecting key: tracksters;1
Subkeys (branches or nested objects):
['event', 'NClusters', 'NTracksters', 'time', 'timeError', 'regressed_energy', 'raw_energy', 'raw_em_energy', 'raw_pt', 'raw_em_pt', 'barycenter_x', 'barycenter_y', 'barycenter_z', 'barycenter_eta', 'barycenter_phi', 'EV1', 'EV2', 'EV3', 'eVector0_x', 'eVector0_y', 'eVector0_z', 'sigmaPCA1', 'sigmaPCA2', 'sigmaPCA3', 'id_probabilities', 'vertices_indexes', 'vertices_x', 'vertices_y', 'vertices_z', 'vertices_time', 'vertices_timeErr', 'vertices_energy', 'vertices_correctedEnergy', 'vertices_correctedEnergyUncertainty', 'vertices_multiplicity']
--------------------------------------------------
Inspecting key: candidates;1
Subkeys (branches or nested objects):
['NCandidates', 'candidate_charge', 'candidate_pdgId', 'candidate_id_probabilities', 'candidate_time', 'candidate_timeErr', 'candidate_energy', 'candidate_raw_energy', 'candidate_px', 'candidate_py', 'candidate_pz', 'track_in_candidate', 'tracksters_in_candidate']
-

In [5]:
track = v_file['clusters']

In [15]:
track_ind = track['cluster_type'].array()

In [16]:
print(track_ind[0])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ... 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5]


In [7]:
print(track_ind[0][2])

ValueError: in ListOffsetArray64 attempting to get 2, index out of range

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.3/src/libawkward/array/ListOffsetArray.cpp#L682)

In [10]:
CP = v_file['simtrackstersCP']
CP_ind = CP['vertices_indexes'].array()

In [11]:
print(CP_ind[0][0])

[75, 103, 63, 92, 52, 118, 39, 134, 28, ... 111, 20, 187, 226, 232, 67, 191, 4, 59]


In [30]:
class CCV1(Dataset):
    r'''
        input: layer clusters

    '''

    url = '/dummy/'

    def __init__(self, root, transform=None, max_events=1e8, inp = 'train'):
        super(CCV1, self).__init__(root, transform)
        self.step_size = 500
        self.inp = inp
        self.max_events = max_events
        self.fill_data(max_events)

    def fill_data(self,max_events):
        counter = 0
        arrLens0 = []
        arrLens1 = []

        print("### Loading data")
        for fi,path in enumerate(tqdm.tqdm(self.raw_paths)):

            if fi > 2:
                break
            if self.inp == 'train':
                cluster_path = 'clusters;3'
                sim_path = 'simtrackstersCP;3'
            else:
                cluster_path = 'clusters;1'
                sim_path = 'simtrackstersCP;1'
            
            crosstree =  uproot.open(path)[cluster_path]
            crosscounter = 0
            for array in uproot.iterate(f"{path}:{sim_path}", ["vertices_x", "vertices_y", "vertices_z", 
            "vertices_energy", "vertices_multiplicity", "vertices_time", "vertices_indexes", "barycenter_x", "barycenter_y", "barycenter_z"], step_size=self.step_size):
            
                tmp_stsCP_vertices_x = array['vertices_x']
                tmp_stsCP_vertices_y = array['vertices_y']
                tmp_stsCP_vertices_z = array['vertices_z']
                tmp_stsCP_vertices_energy = array['vertices_energy']
                tmp_stsCP_vertices_time = array['vertices_time']
                tmp_stsCP_vertices_indexes = array['vertices_indexes']
                tmp_stsCP_barycenter_x = array['barycenter_x']
                tmp_stsCP_barycenter_y = array['barycenter_y']
                tmp_stsCP_barycenter_z = array['barycenter_z']

                #Filtering all the points where energy percent < 50%
                tmp_stsCP_vertices_multiplicity = array['vertices_multiplicity']
                
                self.step_size = min(self.step_size,len(tmp_stsCP_vertices_x))


                # Code block for reading from other tree
                tmp_all_vertices_layer_id = crosstree['cluster_layer_id'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                #tmp_all_vertices_radius = crosstree['cluster_radius'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                tmp_all_vertices_noh = crosstree['cluster_number_of_hits'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                tmp_all_vertices_eta = crosstree['position_eta'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                tmp_all_vertices_phi = crosstree['position_phi'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                crosscounter += 1

                layer_id_list = []
                radius_list = []
                noh_list = []
                eta_list = []
                phi_list = []
                for evt_row in range(len(tmp_all_vertices_noh)):
                    #print("Event no: %i"%evt_row)
                    #print("There are %i particles in this event"%len(tmp_stsCP_vertices_indexes[evt_row]))
                    layer_id_list_one_event = []
                    #radius_list_one_event = []
                    noh_list_one_event = []
                    eta_list_one_event = []
                    phi_list_one_event = []
                    for particle in range(len(tmp_stsCP_vertices_indexes[evt_row])):
                        #print("Particle no: %i"%particle)
                        #print("A")
                        #print(np.array(tmp_all_vertices_radius[evt_row]).shape)
                        #print("B")
                        #print(np.array(tmp_stsCP_vertices_indexes[evt_row][particle]).shape)
                        #print("C")
                        tmp_stsCP_vertices_layer_id_one_particle = tmp_all_vertices_layer_id[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        #tmp_stsCP_vertices_radius_one_particle = tmp_all_vertices_radius[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        tmp_stsCP_vertices_noh_one_particle = tmp_all_vertices_noh[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        tmp_stsCP_vertices_eta_one_particle = tmp_all_vertices_eta[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        tmp_stsCP_vertices_phi_one_particle = tmp_all_vertices_phi[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        #print(tmp_stsCP_vertices_radius_one_particle)
                        layer_id_list_one_event.append(tmp_stsCP_vertices_layer_id_one_particle)
                        #radius_list_one_event.append(tmp_stsCP_vertices_radius_one_particle)
                        noh_list_one_event.append(tmp_stsCP_vertices_noh_one_particle)
                        eta_list_one_event.append(tmp_stsCP_vertices_eta_one_particle)
                        phi_list_one_event.append(tmp_stsCP_vertices_phi_one_particle)
                    layer_id_list.append(layer_id_list_one_event)
                    #radius_list.append(radius_list_one_event)
                    noh_list.append(noh_list_one_event)
                    eta_list.append(eta_list_one_event)
                    phi_list.append(phi_list_one_event)
                tmp_stsCP_vertices_layer_id = ak.Array(layer_id_list)                
                #tmp_stsCP_vertices_radius = ak.Array(radius_list)                
                tmp_stsCP_vertices_noh = ak.Array(noh_list)                
                tmp_stsCP_vertices_eta = ak.Array(eta_list)                
                tmp_stsCP_vertices_phi = ak.Array(phi_list)                

                energyPercent = 1/tmp_stsCP_vertices_multiplicity
                skim_mask_energyPercent = energyPercent > 0.5
                tmp_stsCP_vertices_x = tmp_stsCP_vertices_x[skim_mask_energyPercent]
                tmp_stsCP_vertices_y = tmp_stsCP_vertices_y[skim_mask_energyPercent]
                tmp_stsCP_vertices_z = tmp_stsCP_vertices_z[skim_mask_energyPercent]
                tmp_stsCP_vertices_energy = tmp_stsCP_vertices_energy[skim_mask_energyPercent]
                tmp_stsCP_vertices_time = tmp_stsCP_vertices_time[skim_mask_energyPercent]
                tmp_stsCP_vertices_layer_id = tmp_stsCP_vertices_layer_id[skim_mask_energyPercent]
                #tmp_stsCP_vertices_radius = tmp_stsCP_vertices_radius[skim_mask_energyPercent]
                tmp_stsCP_vertices_noh = tmp_stsCP_vertices_noh[skim_mask_energyPercent]
                tmp_stsCP_vertices_eta = tmp_stsCP_vertices_eta[skim_mask_energyPercent]
                tmp_stsCP_vertices_phi = tmp_stsCP_vertices_phi[skim_mask_energyPercent]
                

                
                #SHOULD BE LEN(E) >= 2 for MULTI particles
                skim_mask = []
                for e in tmp_stsCP_vertices_x:
                    if len(e) == 2:
                        skim_mask.append(True)
                    else:
                        skim_mask.append(False)
                tmp_stsCP_vertices_x = tmp_stsCP_vertices_x[skim_mask]
                tmp_stsCP_vertices_y = tmp_stsCP_vertices_y[skim_mask]
                tmp_stsCP_vertices_z = tmp_stsCP_vertices_z[skim_mask]
                tmp_stsCP_vertices_energy = tmp_stsCP_vertices_energy[skim_mask]
                tmp_stsCP_vertices_time = tmp_stsCP_vertices_time[skim_mask]
                tmp_stsCP_vertices_layer_id = tmp_stsCP_vertices_layer_id[skim_mask]
                #tmp_stsCP_vertices_radius = tmp_stsCP_vertices_radius[skim_mask]
                tmp_stsCP_vertices_noh = tmp_stsCP_vertices_noh[skim_mask]
                tmp_stsCP_vertices_eta = tmp_stsCP_vertices_eta[skim_mask]
                tmp_stsCP_vertices_phi = tmp_stsCP_vertices_phi[skim_mask]

                if counter == 0:
                    self.stsCP_vertices_x = tmp_stsCP_vertices_x
                    self.stsCP_vertices_y = tmp_stsCP_vertices_y
                    self.stsCP_vertices_z = tmp_stsCP_vertices_z
                    self.stsCP_vertices_energy = tmp_stsCP_vertices_energy
                    self.stsCP_vertices_time = tmp_stsCP_vertices_time
                    self.stsCP_vertices_layer_id = tmp_stsCP_vertices_layer_id
                    #self.stsCP_vertices_radius = tmp_stsCP_vertices_radius
                    self.stsCP_vertices_noh = tmp_stsCP_vertices_noh
                    self.stsCP_vertices_eta = tmp_stsCP_vertices_eta
                    self.stsCP_vertices_phi = tmp_stsCP_vertices_phi
                    self.stsCP_vertices_indexes = tmp_stsCP_vertices_indexes
                    self.stsCP_barycenter_x = tmp_stsCP_barycenter_x
                    self.stsCP_barycenter_y = tmp_stsCP_barycenter_y
                    self.stsCP_barycenter_z = tmp_stsCP_barycenter_z
                else:
                    self.stsCP_vertices_x = ak.concatenate((self.stsCP_vertices_x,tmp_stsCP_vertices_x))
                    self.stsCP_vertices_y = ak.concatenate((self.stsCP_vertices_y,tmp_stsCP_vertices_y))
                    self.stsCP_vertices_z = ak.concatenate((self.stsCP_vertices_z,tmp_stsCP_vertices_z))
                    self.stsCP_vertices_energy = ak.concatenate((self.stsCP_vertices_energy,tmp_stsCP_vertices_energy))
                    self.stsCP_vertices_time = ak.concatenate((self.stsCP_vertices_time,tmp_stsCP_vertices_time))
                    self.stsCP_vertices_layer_id = ak.concatenate((self.stsCP_vertices_layer_id,tmp_stsCP_vertices_layer_id))
                    #self.stsCP_vertices_radius = ak.concatenate((self.stsCP_vertices_radius,tmp_stsCP_vertices_radius))
                    self.stsCP_vertices_noh = ak.concatenate((self.stsCP_vertices_noh,tmp_stsCP_vertices_noh))
                    self.stsCP_vertices_eta = ak.concatenate((self.stsCP_vertices_eta,tmp_stsCP_vertices_eta))
                    self.stsCP_vertices_phi = ak.concatenate((self.stsCP_vertices_phi,tmp_stsCP_vertices_phi))
                    self.stsCP_vertices_indexes = ak.concatenate((self.stsCP_vertices_indexes,tmp_stsCP_vertices_indexes))
                    self.stsCP_barycenter_x = ak.concatenate((self.stsCP_barycenter_x,tmp_stsCP_barycenter_x))
                    self.stsCP_barycenter_y = ak.concatenate((self.stsCP_barycenter_y,tmp_stsCP_barycenter_y))
                    self.stsCP_barycenter_z = ak.concatenate((self.stsCP_barycenter_z,tmp_stsCP_barycenter_z))

                #print(len(self.stsCP_vertices_x))
                counter += 1
                if len(self.stsCP_vertices_x) > max_events:
                    print(f"Reached {max_events}!")
                    break
            if len(self.stsCP_vertices_x) > max_events:
                break
     
            
            
    def download(self):
        raise RuntimeError(
            'Dataset not found. Please download it from {} and move all '
            '*.z files to {}'.format(self.url, self.raw_dir))

    def len(self):
        return len(self.stsCP_vertices_x)

    @property
    def raw_file_names(self):
        raw_files = sorted(glob.glob(osp.join(self.raw_dir, '*.root')))
        
        #raw_files = [osp.join(self.raw_dir, 'step3_NTUPLE.root')]

        return raw_files

    @property
    def processed_file_names(self):
        return []


    def get(self, idx):
        edge_index = torch.empty((2,0), dtype=torch.long)
         
        lc_indx = self.stsCP_vertices_indexes
        lc_x = self.stsCP_vertices_x[idx]
        #print(ak.to_numpy(lc_x[0]).shape)
        #print(ak.to_numpy(lc_x[1]).shape)
        flat_lc_x = np.expand_dims(np.array(ak.flatten(lc_x)),axis=1)
        lc_y = self.stsCP_vertices_y[idx]
        flat_lc_y = np.expand_dims(np.array(ak.flatten(lc_y)),axis=1)
        lc_z = self.stsCP_vertices_z[idx]
        flat_lc_z = np.expand_dims(np.array(ak.flatten(lc_z)),axis=1)
        lc_e = self.stsCP_vertices_energy[idx]
        flat_lc_e = np.expand_dims(np.array(ak.flatten(lc_e)),axis=1)     
        lc_t = self.stsCP_vertices_time[idx]
        flat_lc_t = np.expand_dims(np.array(ak.flatten(lc_t)),axis=1)  
        lc_layer_id = self.stsCP_vertices_layer_id[idx]
        flat_lc_layer_id = np.expand_dims(np.array(ak.flatten(lc_layer_id)),axis=1)  
        #lc_radius = self.stsCP_vertices_radius[idx]
        #flat_lc_radius = np.expand_dims(np.array(ak.flatten(lc_radius)),axis=1)  
        lc_noh = self.stsCP_vertices_noh[idx]
        flat_lc_noh = np.expand_dims(np.array(ak.flatten(lc_noh)),axis=1)  
        lc_eta = self.stsCP_vertices_eta[idx]
        flat_lc_eta = np.expand_dims(np.array(ak.flatten(lc_eta)),axis=1)  
        lc_phi = self.stsCP_vertices_phi[idx]
        flat_lc_phi = np.expand_dims(np.array(ak.flatten(lc_phi)),axis=1)  

        #lc_rad = self.stsCP_vertices_radius[idx]
        #flat_lc_rad = np.expand_dims(np.array(ak.flatten(lc_rad)),axis=1)  

        #lc_id = self.stsCP_vertices_id[idx]
        #flat_lc_id = np.expand_dims(np.array(ak.flatten(lc_id)),axis=1)                                                                                          


        #flat_lc_feats = np.concatenate((flat_lc_x,flat_lc_y,flat_lc_z,flat_lc_e),axis=-1)
        flat_lc_feats = np.concatenate((flat_lc_x,flat_lc_y,flat_lc_z,flat_lc_e,\
                                        flat_lc_layer_id,flat_lc_noh,flat_lc_eta,flat_lc_phi),axis=-1)        

        # Loop over number of calo particles to build positive and negative edges
        pos_edges = []
        neg_edges = []
        offset = 0
        idlc = 0
        for cp in range(len(lc_x)):
            n_lc_cp = len(lc_x[cp])
            
            # First the positive edge
            for lc in range(n_lc_cp):

                random_num_pos = int(random.uniform(offset, offset+n_lc_cp))
                random_num_neg = int(random.uniform(0, flat_lc_x.shape[0]))
                while random_num_neg >= offset and random_num_neg < offset+n_lc_cp:
                    random_num_neg = int(random.uniform(0, flat_lc_x.shape[0]))

                #while (idlc == random_num_pos):
                #    random_num_pos = int(random.uniform(offset, offset+n_lc_cp))
                pos_edges.append([idlc,random_num_pos])
                neg_edges.append([idlc,random_num_neg])
                idlc += 1
            offset += n_lc_cp

        x = torch.from_numpy(flat_lc_feats).float()
        x_lc = x
        x_pos_edge = torch.from_numpy(np.array(pos_edges))
        x_neg_edge = torch.from_numpy(np.array(neg_edges))       

        x_counts = lc_x
        y = torch.from_numpy(np.array([0 for u in range(len(flat_lc_feats))])).float()
        return Data(x=x, edge_index=edge_index, y=y,
                        x_lc=x_lc, x_pe=x_pos_edge, x_ne=x_neg_edge, x_counts = x_counts, x_ind = lc_indx)


In [31]:
ipath = "/vols/cms/mm1221/Data/2e/val"

In [32]:
data_train = CCV1(ipath, max_events=1, inp = 'val')

### Loading data


  0%|                                                     | 0/1 [00:12<?, ?it/s]

Reached 1!


In [33]:
print(data_train[0].x_pe)

tensor([[  0, 158],
        [  1, 123],
        [  2,  73],
        [  3,   9],
        [  4,  57],
        [  5, 132],
        [  6, 158],
        [  7, 159],
        [  8, 123],
        [  9,  73],
        [ 10, 119],
        [ 11, 117],
        [ 12,  31],
        [ 13, 143],
        [ 14,  81],
        [ 15, 121],
        [ 16, 166],
        [ 17,  69],
        [ 18,  84],
        [ 19,  80],
        [ 20, 137],
        [ 21, 139],
        [ 22, 150],
        [ 23,  28],
        [ 24,  53],
        [ 25, 163],
        [ 26, 130],
        [ 27, 132],
        [ 28, 169],
        [ 29, 144],
        [ 30,  93],
        [ 31, 125],
        [ 32, 142],
        [ 33, 104],
        [ 34,   9],
        [ 35, 156],
        [ 36, 165],
        [ 37,  95],
        [ 38,  22],
        [ 39, 171],
        [ 40,  46],
        [ 41,  72],
        [ 42,  62],
        [ 43,  16],
        [ 44,  43],
        [ 45, 129],
        [ 46,  30],
        [ 47,  70],
        [ 48,  33],
        [ 49, 137],


In [45]:
print(data_train[0].x_ind[0][1])

[59, 49, 71, 33, 27, 100, 18, 86, 115, 129, ... 26, 69, 23, 16, 65, 279, 54, 0, 37]


In [46]:
# Combine all tracksters into a single list and sort
tracksters = data_train[0].x_ind[0]  # List of tracksters, each containing global indices
global_indices = sorted([index for trackster in tracksters for index in trackster])  # Flatten and sort

# Map pos_edge indices from local to global
pos_edge_local = data_train[0].x_pe.numpy()  # Convert PyTorch tensor to numpy
pos_edge_global = [(global_indices[i], global_indices[j]) for i, j in pos_edge_local]

# Verify that each pair in pos_edge belongs to the same trackster
valid_pairs = []
for u, v in pos_edge_global:
    # Check if both vertices belong to the same trackster
    in_same_trackster = any(u in trackster and v in trackster for trackster in tracksters)
    valid_pairs.append((u, v, in_same_trackster))

# Count and print mismatched pairs
mismatched_pairs = [(u, v) for u, v, is_valid in valid_pairs if not is_valid]
print(f"Number of mismatched pairs: {len(mismatched_pairs)}")
if mismatched_pairs:
    print(f"Mismatched pairs: {mismatched_pairs}")
else:
    print("All pairs are valid.")

Number of mismatched pairs: 86
Mismatched pairs: [(0, 57), (2, 143), (4, 73), (5, 23), (6, 53), (17, 56), (18, 85), (19, 26), (24, 133), (29, 69), (34, 135), (38, 146), (40, 113), (42, 73), (43, 97), (47, 9), (53, 25), (54, 142), (55, 35), (58, 55), (60, 79), (66, 131), (68, 61), (77, 18), (89, 143), (90, 109), (93, 7), (95, 114), (97, 122), (106, 116), (107, 131), (108, 32), (113, 124), (114, 5), (116, 21), (119, 10), (123, 146), (125, 17), (130, 141), (135, 32), (136, 99), (140, 101), (144, 12), (146, 109), (147, 135), (150, 116), (153, 90), (156, 177), (157, 177), (159, 182), (160, 253), (164, 240), (166, 206), (167, 192), (169, 245), (173, 244), (174, 211), (176, 205), (178, 166), (182, 164), (185, 172), (190, 206), (192, 179), (193, 238), (194, 220), (195, 188), (199, 157), (201, 248), (204, 209), (206, 195), (207, 254), (208, 193), (209, 169), (218, 160), (220, 177), (226, 227), (228, 214), (232, 159), (233, 213), (242, 217), (247, 159), (248, 218), (250, 171), (253, 195), (255, 